## Identify Points for StreamStats

__Description__: Tool to identify confluence pair points for tributaries of a specific length, add points to the main stem of a stream network at a specific distance interval, and export a shapefile of the points.

__Input__: Stream grid from the [SteamStats Repository](https://streamstatsags.cr.usgs.gov/StreamGrids/directoryBrowsing.asp), masked using `ClipRaster_withMask.ipynb` and the latitude and longitude of the catchment outlet.

__Output__: A shapefile containing the latitude and longitude of points on the stream grid (confluence and main stem locations).


*Authors*: sputnam@Dewberry.com & slawler@Dewberry.com

### Load libraries and Python options:

In [1]:
import os
import sys
sys.path.append('../USGStools')
from StreamStats_ID_Points import*
import ID_Confluences
import Calc_Distance

### Load the clipped stream grid, specify the catchment outlet, and the tributary exclusion distance:

##### Specify:

In [63]:
path=r'C:\Users\tmiesse\Work\ryans_data' #Specify the location of the stream grid raster which was masked by the catchment polygon
name='orleans_streams2.tif' #The name of the stream grid raster

lat = 4807760.7#[m]; latitude of the catchment outlet
lon = 229158.4 #[m]; longitude of the catchment outlet 

disexl=(5280/2.0)*(0.3048) #[m]; Tributary exclusion distance (0.5 miles)

##### Load the stream grid:

In [64]:
sg = StreamGrid(os.path.join(path, name)) #Open the stream grid raster and create an object

crs=sg.crs_value() #Extract the coordinate reference system value (epsg) for the raster
print("epsg:",crs) 

df = sg.dataframe() #Create a dataframe from the stream grid data

df.replace(255, 0, inplace=True) #Potential Issue: Replace 255 with 0, where 255 corresponds to the non-stream cells
df.head(n=2) 

epsg: 26918


,0,1,2,3,4,5,6,7,8,9,...,42714,42715,42716,42717,42718,42719,42720,42721,42722,42723
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### Convert the catchment outlet's lat/lon to row/column in the stream grid dataframe and extract the cell size:

In [65]:
pix_x, pix_y =coord2index(sg, lat, lon) #Transform the lat and lon values to the row/column location with the stream grid dataframe
pourpoint=[(pix_x, pix_y)] #Add these values to a list as a touple
print("Pourpoint XY:", pourpoint)

cellsize=sg.cell_size() # [m]; Raster cell size
print("The Cell Size:", cellsize)

Pourpoint XY: [(1328, 5413)]
The Cell Size: 9.999999999999998


### Move up the stream and identify the confluences:

In [66]:
save_confluence, confluence_pairs_orig, nogo=ID_Confluences.main(pourpoint, df,display=True) #Function that moves up a stream network from the pourpoint and identifies the confluence pairs

print("Number of points identified:", len(save_confluence))

Confluence Number: 100
Number of points identified: 106


### Remove superflous confluences:

In [67]:
#A false confluence is where there are not two points with the same confluence number
true_confluence, false_confluence=Remove_False_Confluence(save_confluence) 
print("Number of true confluence points:", len(true_confluence))

Number of true confluence points: 91


##### Identify all stream cells associated with the false confluences:

In [68]:
false_points=ID_False_ConfluenceLocs(false_confluence, nogo) 

##### Remove any original confluences associated with the false confluences:

In [69]:
confluence_pairs_orig=Remove_False_From_Orig(false_confluence, confluence_pairs_orig) 

### Calculate the tributary length:

In [70]:
tributary, mainstem, nogoabs=Calc_Distance.main(df, cellsize, true_confluence, false_points, confluence_pairs_orig) 

print("Total Confluence Points:", (len(mainstem)+len(tributary)), "Main Stem Points:", len(mainstem))

Total Confluence Points: 91 Main Stem Points: 46


##### Remove confluences with tributaries or intervals less than a specific length:

In [71]:
incl_tribs=Exclude_Confls(tributary, disexl) 

print("# of Tributary Points:", len(incl_tribs)) 

 #Note that this is a temporary fix until the distance calculation is fixed so that it adds up excluded tribs.
incl_mainstem=Exclude_Confls(mainstem, disexl/2.) 
 #Add in the original catchment outlet
incl_mainstem.append((pourpoint[0]+(0,0,)))

print("# of Main Stem Points:", len(incl_mainstem))   

# of Tributary Points: 2
# of Main Stem Points: 19


##### Add labels to differentiate between the mainstem and the tributaries and combine:

In [72]:
mainstem_withlabel=[] #Empty list to store the labeled main stem points
tributary_withlabel=[] #Empty list to store the labeled tributary points 

for cell in incl_mainstem: #Add a "M" to the tuple to indicate that the cell is a main stem location
    mainstem_withlabel.append(cell+('M',))
    
for cell in incl_tribs: #Add a "T" to the tuple to indicate that the cell is a tributary location
    tributary_withlabel.append(cell+('T',))
    
export_confluences=mainstem_withlabel+tributary_withlabel #Combine the main stem and tributary lists into a singe list    

### Save the results:

##### Extract the confluence number, distance, and type of confluence:

In [73]:
export_cnum=[] #Empty list to store the confluence number
export_dis=[] #Empty list to store the distance
export_type=[] #Empty list to store the point type

for cell in export_confluences: #Store the confluence number, distance, and point type.
    export_cnum.append(cell[2])
    export_dis.append(cell[3])    
    export_type.append(cell[4])

##### Transform and save as a shapefile:

In [74]:
longitude, latitude=index2coord(sg, export_confluences)  #Transform the row/column value to latitude/longitude for each confluence

gdf=geodataframe(longitude, latitude, crs, export_cnum, export_dis, export_type) #Store the longitude/latitude, confluence number, distance, and type for each confluence in a geodataframe

gdf.to_file(filename = os.path.join(path,'misssing5.shp')) #Export the geodataframe as a shapefule

# End